*Основная задача*: Зарегистрируйтесь на платформе Kaggle и скачайте набор данных, и по образцу создайте нейронную сеть. Выведите матрицу ошибок и постарайтесь минимизировать их, подобрав параметры нейронной сети.

In [1]:
from tensorflow.keras.datasets import mnist

ModuleNotFoundError: No module named 'tensorflow.python'

*1. Загрузите данные с площадки Kaggle.*

*2. Подготавливаем данные (приводим их к нужным типам данных, проводим базовое EDA, делаем выводы, если возможно).*

*3. Создаем тренировочную и валидационную выборки.*

*4. Масштабируем данные известными вам методами из библиотеки Scikit-learn.*

*5. Создайте функцию, при вызове которой будет собираться и компилироваться (model.compile) архитектура модели. Функция должна возвращать собранную архитектуру. Параметры слоев и их количество вы выбираете сами, стараясь добиться наилучшего качества.*

*6. Обучите модель.*

*7. Оцените качество модели по выбранной вами метрике (выбор метрики должен быть обоснован).*

*8. Постройте графики точности и графики ошибки в зависимости от количества эпох для тренировочной выборки и для валидационной.*

*9. Постройте матрицу ошибок для валидационной выборки.*

ВЫВОД:

# ЗАДАНИЕ ПРО (Не обязательно к выполнению):
Оптимизируйте параметры модели в автоматическом режиме. Можно использовать алгоритмы оптимизации типа GridSearch, но не сильно увлекаясь — иначе расчёт может быть излишне долог.

*1. Установите библиотеку scikeras, либо найдите как импортировать модуль KerasRegressor в tensorflow.*

*2. Из этой же библиотеки scikeras импортируйте модуль KerasRegressor (найдите в документации как его импортировать).*

*3. Поместите вашу модель в оболочку KerasRegressor.*

*4. Подберите оптимальные параметры для модели с помощью GridSearchCV или RandomizedSearchCV.*

*5. Посчитайте метрику с наилучшими параметрами, напишите наилучшие параметры для модели в выводе.*

**ОБЩИЙ ВЫВОД:**